# Assignment 3:  Decision Tree Implementation
*Margaret Thomann - February 17, 2018 *

In this assignment, I will construct a decision tree from the data provided about heart disease.

### Reading the data and assigning counts to arrays and Data class

#### Data Class 
A Data class will be instantiated for each line of the data.  It will then be added to one of two arrays (explained later).

In [3257]:
from collections import OrderedDict

class Data:
    def __init__(self, class_value):
        self.class_value = class_value
        self.data_vars = OrderedDict()

### Calculate Information Gain for Each Feature
The below function can be used to determine the information gain for a given data and hypothesis (passed in as a string - x and y).  Information Gain can be represented as: Infgain(Y|X_K) = H(Y) - H(Y|X_K)

In [3258]:
# Arrays for the Data instances
#     absence_heart_array  : contains all Data instantiations where heart disease is absent
#     presence_heart_array : contains all Data instantiations where heart disease is absent
absence_heart_array = []
presence_heart_array = []
total_data_array = []

# Classify the features according to their type: nominal or continuous
indices_for_nominal = [10, 1, 5, 8, 6, 2, 12]
indices_for_continuous = [0,3,4,7,9,11]
feature_names = ["age", "sex", "chest_pain_type", "resting_blood_pressure", "serum_cholesterol", "fasting_blood_sugar",
                "resting_electrocardiographic_results", "maximum_heart_rate_achieved", "exercise_induced_angina",
                "oldpeak", "slope_peak_exercise", "number_of_major_vessels", "thal", "has_heart_disease"]
features_and_types = OrderedDict()
for feature in feature_names:
    if feature_names.index(feature) in indices_for_continuous:
        features_and_types[feature] = "continuous"
    else:
        features_and_types[feature] = "nominal"

# Process the data and store it in the arrays
data = open('heart.data.txt')
for line in data.readlines():
    feature_value_list = line.split()
    has_heart_disease = int(feature_value_list[-1])
    data = Data(has_heart_disease)
    counter = 0
    feature_dict = OrderedDict()
    for feature in feature_value_list:
        data.data_vars[feature_names[counter]] = float(feature)
        counter += 1
    if has_heart_disease == 2:
        presence_heart_array.append(data)
    elif has_heart_disease == 1:
        absence_heart_array.append(data)
    total_data_array.append(data)

presence_heart_array_num = len(presence_heart_array)
absence_heart_array_num = len(absence_heart_array)
print "✔ Data processed"
print "-----------------"
print "\t" + str(presence_heart_array_num) + " = # Of People with Heart Disease"
print "\t" + str(absence_heart_array_num) + " = # Of People without Heart Disease"

✔ Data processed
-----------------
	120 = # Of People with Heart Disease
	150 = # Of People without Heart Disease


### Utility Printer Function to Print out Data
The below function can be called with a Data object as its input to print out that data objects contents.

In [3259]:
def print_data(data_obj):
    print "_____________"
    for feature in data_obj.data_vars.keys():
        print data_obj.data_vars[feature], ": ", feature

In [3260]:
import math

def compute_info_gain(y, y_values, x, buckets, considered_data):
        
    '''
    if x == "sex":
        print "sex vals!!!"
        for data_point in considered_data:
            print data_point.data_vars["sex"]
        #print "y_values:"
        #print y_values
    '''
    
    
    '''
    if y == "chest_pain_type":
        print "y_values:"
        print y_values
    '''
    
    #print "Calling compute_info_gain with y = "+y+" and x = "+x
    
        
    # Define dicts for the counts
    positive_y_counts = {}
    positive_x_counts = {}
    negative_y_counts = {}
    negative_x_counts = {}
    
    # Get the bucket values
    for bucket in buckets:
        # Convert to string
        s = ""
        for num in list(set(bucket)):
            s += (str(num)+ " ")
        positive_x_counts[s] = 0
        negative_x_counts[s] = 0
    
    
    for val in y_values:
        positive_y_counts[str(float(val))] = 0
        negative_y_counts[str(float(val))] = 0
    
    y_denom = 0
    for data in considered_data:
        y_denom += 1
        has_heart_disease = data.data_vars["has_heart_disease"]
        y_value = data.data_vars[y]
        x_value = data.data_vars[x]
        array_x = []
        array_y = []
        if has_heart_disease == 1:
            # Update all of the negative arrays
            array_x = negative_x_counts
            array_y = negative_y_counts
        elif has_heart_disease == 2:
            # Update all of the positive arrays
            array_x = positive_x_counts
            array_y = positive_y_counts
            
        # Value is not in dictionary yet
        # so set the occurence for that value to 1
        '''
        if value not in array_y.keys():
            array_y[y_value] = 1 
        # Value is already in dictionary
        # so increase the occurence count for that value by 1
        else:
            current_count_for_value = array_y[y_value]
            array_y.update({y_value:current_count_for_value+1})
        '''
        for key in array_x.keys():
            if str(x_value)+" " in key:
                current_count_for_value = array_x[key]
                array_x.update({key:current_count_for_value+1})
                
        for key in array_y.keys():
            if str(y_value) in key:
                current_count_for_value = array_y[key]
                array_y.update({key:current_count_for_value+1})

    '''
    y_denom = 0
    
    for data in presence_heart_array:
        y_denom += 1
        value = data.data_vars[y]
        
        # Value is not in dictionary yet
        # so set the occurence for that value to 1
        if value not in positive_y_counts.keys():
            positive_y_counts[value] = 1 
        # Value is already in dictionary
        # so increase the occurence count for that value by 1
        else:
            current_count_for_value = positive_y_counts[value]
            positive_y_counts.update({value:current_count_for_value+1})
            
        # Same thing is done for processing x:
        x_value = data.data_vars[x]
        for key in positive_x_counts.keys():
            if str(x_value)+" " in key:
                current_count_for_value = positive_x_counts[key]
                positive_x_counts.update({key:current_count_for_value+1})
            
    
    for data in absence_heart_array:
        y_denom += 1
        value = data.data_vars[y]
        
        # Value is not in dictionary yet
        # so set the occurence for that value to 1
        if value not in negative_y_counts.keys():
            negative_y_counts[value] = 1 
        # Value is already in dictionary
        # so increase the occurence count for that value by 1
        else:
            current_count_for_value = negative_y_counts[value]
            negative_y_counts.update({value:current_count_for_value+1})
            
        # Same thing is done for processing x:
        x_value = data.data_vars[x]
        for key in negative_x_counts.keys():
            if str(x_value)+" " in key:
                current_count_for_value = negative_x_counts[key]
                negative_x_counts.update({key:current_count_for_value+1})
                
    '''
    '''
    if x == "sex":
        print "postive_y_counts array"
        print positive_y_counts
        print "negative_y_counts array"
        print negative_y_counts
    '''   
            
    h_of_y = 0
    for count in positive_y_counts.values():
        '''
        if y == "chest_pain_type":
            print "positive_y_count: "+str(count)
        '''
        
        entropy = 0
        if count != 0:
            p = float(float(count)/float(y_denom))
            entropy = -1 * p * (math.log(p, 2))
        h_of_y += entropy
    for count in negative_y_counts.values():

        '''
        if y == "chest_pain_type":
            print "negative_y_count: "+str(count)
        '''
    
        entropy = 0
        if count != 0:
            p = float(float(count)/float(y_denom))
            entropy = -1 * p * (math.log(p, 2))
        h_of_y += entropy
    
    h_of_y_given_x = 0
    for feature_value in positive_x_counts.keys():
        '''
        if x == "sex":
            print "positive_x_counts: "+str(positive_x_counts[feature_value])+" for feature value: "+str(feature_value)
            print "negative_x_counts: "+str(negative_x_counts[feature_value])+" for feature value: "+str(feature_value)        
        '''
        entropy_positive = 0
        entropy_negative = 0
        sum_of_values = positive_x_counts[feature_value] + negative_x_counts[feature_value] 
        fraction = float(float(sum_of_values)/float(y_denom))
        if positive_x_counts[feature_value] != 0:
            p_positive = float(float(positive_x_counts[feature_value])/float(sum_of_values)) 
            entropy_positive = p_positive * (math.log(p_positive, 2))
        if negative_x_counts[feature_value] != 0:
            p_negative = float(float(negative_x_counts[feature_value])/float(sum_of_values)) 
            entropy_negative = p_negative * (math.log(p_negative, 2))
        
        h_of_y_given_x += fraction*(entropy_positive+entropy_negative)
    
    '''
    if x == "sex":
        print "h_of_y:"+str(h_of_y)
        print "h_of_y_given_x:"+str(h_of_y_given_x)
    '''
    
    '''
    if h_of_y_given_x < 0:
        info_gain = h_of_y - (h_of_y_given_x)
        return info_gain
    else:
        info_gain = h_of_y - (-1*h_of_y_given_x)
        return info_gain
    '''
    #print "h of y: ", h_of_y
    #print "h of y given x: ", h_of_y_given_x
    info_gain = h_of_y + h_of_y_given_x
    return info_gain


### Utility Function used to generate all of the possible splits
Credit:  https://stackoverflow.com/questions/19368375/set-partitions-in-python/30134039#30134039

In [3261]:
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller

### Determine possible splits
These will be used for the information gain

In [3262]:
# get_continuous_binary_split(feature):     
#    Gets a feature and splits the data for that feature in all the possible ways to split that
#    data into two buckets.  It returns a dictionary where the key is the number it split on and the
#    value is a list of two lists.  The first element of the list is all of the elements less than
#    the split and the second element of the list is a list of all of the elements greater than or
#    equal to the split.
#    For example:
#           feature = "age"
#           splits_dict = {50: [[20,40,43...],[50,60,61...]], 60: [[57,45,59...],[60,61,63...]]}
def get_continuous_binary_split(feature, considered_data):
    # Ensure the function is being called only with continuous features
    if features_and_types[feature] != "continuous":
        raise ValueError('Error in get_continuous_binary_split: input feature is not continuous.')
        return
    
    # Create a list of the possible splits
    splits = []
    counts_for_feature_values = {}
    
    print "The Length of the Considered Data: ",len(considered_data)
    
    for data in considered_data:
        feature_value = float(data.data_vars[feature])
        if feature_value not in splits:
            splits.append(feature_value)
        if feature_value not in counts_for_feature_values:
            counts_for_feature_values[feature_value] = 1
        else:
            current = counts_for_feature_values[feature_value]
            counts_for_feature_values[feature_value] = current + 1
    
    # Process the splits and create the less than list and greater than or equal to list for each
    splits_list = []
    for split in splits:
        lt_split_feature_values = []
        gtequal_split_feature_values = []
        for data in considered_data:
            feature_value = float(data.data_vars[feature])
            if feature_value < split:
                lt_split_feature_values.append(feature_value)
            else:
                gtequal_split_feature_values.append(feature_value)
        #if len(lt_split_feature_values) != 0:
        splits_list.append([lt_split_feature_values, gtequal_split_feature_values])
            
    # Generate a list of partition dicts
    # Each list element will be a list of dictionaries
    # The dictionaries will contain the feature_value and the number of times it occurs
    # The list of these dictionaries will be split up by partition
    partitions_and_values = []
    for partition in splits_list:
        split_inclusive = []
        for l in partition:
            set_l = set(l)
            count_dict = {}
            for feature_value in set_l:
                num_of_feature_value_occurences = counts_for_feature_values[feature_value]
                count_dict[feature_value] = num_of_feature_value_occurences
            split_inclusive.append(count_dict)
        partitions_and_values.append(split_inclusive)
            
    return partitions_and_values

In [3263]:
import itertools
# get_nominal_split(feature):     
#    Gets a nominal feature and splits the data for that feature in all the possible ways to split that
#    data into every possible number of buckets.  It returns a dictionary where the key is the number it 
#    split on and the value is a list of the lists it generated from the split.  

def split_list(data, n):
    from itertools import combinations, chain
    for splits in combinations(range(1, len(data)), n-1):
        result = []
        prev = None
        for split in chain(splits, [None]):
            result.append(data[prev:split])
            prev = split
        yield result
        
def get_nominal_split(feature, considered_data):
    # Ensure the function is being called only with nominal features
    if features_and_types[feature] != "nominal":
        raise ValueError('Error in get_nominal_split: input feature is not nominal.')
        return
    
    #print "get_nominal_split called with feature: "+feature
    # Determine the unique values for the feature and the counts for the feature_value
    splits = {}
    split_nums = []
    for data in considered_data:
        feature_value = float(data.data_vars[feature])
        if feature_value not in splits:
            split_nums.append(feature_value)
            splits[feature_value] = 1
        else:
            current_num = splits[feature_value]
            splits[feature_value] = current_num+1
    
    #print "possible values for feature:"
    #print split_nums

    # Generate all the possible ways to partition the numbers  
    # possible_partitions is a list of tuples where the first element of the tuple
    # is the kth possible partition and the second element of the tuple is a list
    # of lists of partitions
    possible_partitions = []
    for p in partition(split_nums):
        # Be sure to remove the split where all values are placed in one bucket
        if (len(sorted(p)) != 1) or ((len(sorted(p)) == 1) and (len(split_nums) == 1)):
            possible_partitions.append(sorted(p))
    #print "possible_partitions:"
    #print possible_partitions
                
    # Generate a list of partition dicts
    # Each list element will be a list of dictionaries
    # The dictionaries will contain the feature_value and the number of times it occurs
    # The list of these dictionaries will be split up by partition
    partitions_and_values = []
    for split_lists in possible_partitions:
        split_dicts = []
        for bucket in split_lists:
            bucket_dict = {}
            for feature_value in bucket:
                num_of_feature_value_occurences = splits[feature_value]
                bucket_dict[feature_value] = num_of_feature_value_occurences
            split_dicts.append(bucket_dict)
        partitions_and_values.append(split_dicts)
    return partitions_and_values

### Compute the best split
Checks the information gain for all the possible splits for a certain feature and reports the best one.

In [3264]:
def compute_best_split(y, y_values, feature, considered_data):
    # Check if the feature is nominal or continuous and split accordingly
    splits = []
    if features_and_types[feature] == "nominal":
        
        splits = get_nominal_split(feature, considered_data)
    else:
        splits = get_continuous_binary_split(feature, considered_data)
    
    # Create a dictionary where the key is the information gain from a particular
    # split and the value is that particular split
    info_gains = {}
    for split in splits:
        info_gain = compute_info_gain(y, y_values, feature, split, considered_data)
        info_gains[info_gain] = split
        
    # Process the dictionary and determine the highest info gain
    #print "info-gains"
    #print "compute_best_info_gain called with: "+feature
    #print "considered_data"
    #print considered_data
    #print info_gains
    max_info_gain = max(info_gains, key=float)
    split_yielding_max_info_gain = info_gains[max_info_gain]
    
    # Print the maximum info gain and corresponding split
    #print "FEATURE: "+feature
    #print "Greatest Info Gain is: "+str(max_info_gain)
    #print "from split:"
    #for split in split_yielding_max_info_gain:
        #print sorted(set(split))
    return max_info_gain, split_yielding_max_info_gain

### Tree Class
Set up classes for the tree.

In [3265]:
class Node(object):
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)

### Split Data
Split the data and return a list of arrays of the split data given the feature and the splits

In [3266]:
def split_data(feature, splits, considered_data):
    #print "Splitting data for feature: "+feature
    print "split is: "
    print splits
    data = []
    for split in splits:
        data.append([])
    separate_buckets = {}
    for data in considered_data:
        feature_value = float(data.data_vars[feature])
        if feature_value not in separate_buckets.keys():
            new_list = []
            new_list.append(data)
            separate_buckets[feature_value] = new_list
        else:
            current_list = separate_buckets[feature_value]
            current_list.append(data)
            separate_buckets[feature_value] = current_list
        
    #splits = [{3.0: 152}, {6.0: 14}, {7.0: 16}]
    #OR splits = [{3.0: 152}, [{6.0: 14}, {7.0: 16}]]
            
    new_data = []
    for split in splits:
        
        print "Processing split: ", split
        
        # Convert it to a list
        split_list = []
        if len(split) == 1:
            
            split_list.append(split)
        else:
            split_list.append(split)
            '''
            for each_value in split:
                print "Appending each value: ", each_value
                split_list.append(each_value)
            '''
        
        # Prepare a list for the data of that split
        data_for_split = []
        
        # Get all the valid feature_values for that particular split
        valid_feature_values = []
        print "SPLIT LIST IN SPLIT DATA: ", split_list
        for split_dict in split_list:
            for feature_value_key in split_dict.keys():
                valid_feature_values.append(feature_value_key)
        
        # Process the valid feature values, find matching data, and add it to the data for that split
        for feature_value in valid_feature_values:
            data_for_split.append(separate_buckets[feature_value])
        
        # Add the data relevant for a certain split group to the new data array
        new_data.append(data_for_split)
        
        
    return new_data

### Recurse to build the tree
Used the information gain function to determine the best splits for each node of the tree.

In [3267]:
def build_tree(y, y_values, data, allowable_features_to_split_on):
    info_gains = {}
    split_yielding_max_info_gain = []
    for feature in allowable_features_to_split_on:
        if feature != y:
            info_gain, split_yielding_max_info_gain = compute_best_split(y, y_values, feature, data)
            info_gains[feature] = [info_gain, split_yielding_max_info_gain]

    #print info_gains

    # Printing out the information gains for each feature
    for feature in info_gains.keys():
        print str(info_gains[feature][0])+" : "+feature

    # Find the maximum in the info_gains returned
    max_info_gain = 0
    best_split = []
    best_feature = ""
    
    # Check if they are all equal to 0
    counter = 0
    total = 0
    for feature in info_gains.keys():
        if info_gains[feature][0] == 0:
            counter += 1
        total += 1
    if counter == total:
        return
    
    #print info_gains.keys()
    for feature in info_gains.keys():
        if info_gains[feature][0] > max_info_gain:
            max_info_gain = info_gains[feature][0]
            best_split = info_gains[feature][1]
            best_feature = feature

    #print max_info_gain
    #print best_feature
    # Split on that feature and optimal split
    new_data = split_data(best_feature, best_split, data)

    # Define the new y based on this data
    y = feature
    
    #print best_split
    return best_feature, new_data, best_split



    # Now we have to feed that new data into the tree and branch off into the leaves
    # so we have three groups for thal which means it should branch into three leaves:
    # one with 3, one with 6, and one with 7.
    # And recursively continue branching
    
    
    
            


In [3268]:
y = "has_heart_disease"
allowable_features_to_split_on = data.data_vars.keys()

feature, new_data, best_split = build_tree(y, [2, 1], total_data_array, allowable_features_to_split_on)
levels = []
levels.append([feature, new_data, best_split])
print best_split
print "Length of new data ", len(new_data)
possible_y_vals_array = []
for split in best_split:
    new_list = []
    for y_val in split.keys():
        new_list.append(y_val)
    possible_y_vals_array.append(new_list)
    
# Edit the features allowed to split on

'''
for data in new_data[1]:
    print data.data_vars["sex"]
'''

allowable_features_to_split_on.remove("has_heart_disease")
    



# Process each feature 
# level = [0: feature, 1: new_data, 2: best_split]
counter = 0
for level in levels:
    feature = level[0]
    for level in levels:
        print level[0]
    print "Processing feature: ", level[0]
    new_data = level[1]
    best_split = level[2]
    print 'Length of New Data',len(new_data)
    
    # Process each split for the feature
    for branch_index in range(len(new_data)):
        print "Processing the branch for:"
        print "\t Feature = ", level[0]
        print "\t Split   = ", level[2][branch_index]
        # Edit the allowable features to split on since if you split on a nominal feature and only have 
        # one value for that nominal feature in the branch, you shouldn't split on it again
        if features_and_types[feature] == "nominal" and len(possible_y_vals_array[0]) == 1:
            allowable_features_to_split_on.remove(level[0])
        # Call build_tree for next partition
        #print "Allowable_features_to_split_on:"
        #print allowable_features_to_split_on
        #print "y is : "+feature
        if len(allowable_features_to_split_on) != 0:
            #print "Possible_y_vals:"
            #print possible_y_vals_array
            #print "branch index: "+str(branch_index)
            
            
            data_for_next_call = new_data[branch_index][0]
            
            possible_y_values = []
            for pt in data_for_next_call:
                    y_value = pt.data_vars[level[0]]
                    if y_value not in possible_y_values:
                        possible_y_values.append(y_value)
                    
            #print "Allowable features to split on:"
            #print allowable_features_to_split_on
            #print "Level[0]"
            #print level[0]
            #print "Possible y values:"
            #print possible_y_values
            #print "new data of branch index:"
            #print new_data[branch_index]
            
            if level[0] == "chest_pain_type":
                for data in data_for_next_call:
                    print_data(data)

            next_feature, next_new_data, next_best_split = build_tree(level[0], possible_y_values, data_for_next_call, allowable_features_to_split_on)
            
            print "*** Next Feature:"
            print next_feature
            #print "*** Next New Data:"
            #print next_new_data
            print "*** Next Best Split:"
            print next_best_split
            print "*** Length of Next New Data:"
            print len(next_new_data)
            levels.append([next_feature, next_new_data, next_best_split])


            # Put feature back in allowable features because it's possible we might need it later
            allowable_features_to_split_on.append(level[0])
    levels.remove(level)
    for e in levels:
        print "Level: ",e[0]
        print "Length of Data: ",len(e[1])
        print "Split: ",e[2]
    
print counter

The Length of the Considered Data:  270
The Length of the Considered Data:  270
The Length of the Considered Data:  270
The Length of the Considered Data:  270
The Length of the Considered Data:  270
The Length of the Considered Data:  270
0.10007075195 : maximum_heart_rate_achieved
0.129914712749 : exercise_induced_angina
0.0164273041343 : resting_blood_pressure
0.0567263200991 : age
0.00019260538564 : fasting_blood_sugar
0.0267986294625 : serum_cholesterol
0.0668955964785 : sex
0.119648312513 : oldpeak
0.165916419313 : number_of_major_vessels
0.111153056361 : slope_peak_exercise
0.19220234895 : chest_pain_type
0.0241518516218 : resting_electrocardiographic_results
0.208555678313 : thal
split is: 
[{3.0: 152}, {6.0: 14}, {7.0: 104}]
Processing split:  {3.0: 152}
SPLIT LIST IN SPLIT DATA:  [{3.0: 152}]
Processing split:  {6.0: 14}
SPLIT LIST IN SPLIT DATA:  [{6.0: 14}]
Processing split:  {7.0: 104}
SPLIT LIST IN SPLIT DATA:  [{7.0: 104}]
[{3.0: 152}, {6.0: 14}, {7.0: 104}]
Length of ne

TypeError: 'NoneType' object is not iterable